<a href="https://colab.research.google.com/github/Akinleyejoshua/-chatbot/blob/main/Deriv_Bot_1_deployed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import asyncio
import websockets
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import logging

warnings.filterwarnings('ignore')

class DerivTradingBot:
    def __init__(self):
        # Connection attributes
        self.app_id = None
        self.token = None
        self.websocket = None

        # Trading parameters
        self.symbol = None
        self.duration = None
        self.duration_unit = None  # 't' for ticks, 's' for seconds
        self.stake_amount = None
        self.max_trades = None
        self.trades_per_signal = None

        # Market condition thresholds
        self.volatile_threshold = None
        self.confidence_threshold = None
        self.min_confidence = None

        # Data management
        self.historical_data = pd.DataFrame()
        self.scaler = StandardScaler()
        self.model = None

        # Trading state
        self.active_trades = []
        self.trade_history = []
        self.balance = 0.0
        self.current_tick = None
        self.trade_count = 0
        self.is_trading = False

        # Statistics
        self.wins = 0
        self.losses = 0
        self.total_profit = 0.0

    def set_connection_params(self, app_id: str, token: str):
        """Set API connection parameters"""
        self.app_id = app_id
        self.token = token

    def set_trading_params(self, symbol: str, duration: int, duration_unit: str,
                          stake_amount: float, max_trades: int, trades_per_signal: int):
        """Set trading parameters"""
        self.symbol = symbol
        self.duration = duration
        self.duration_unit = duration_unit
        self.stake_amount = stake_amount
        self.max_trades = max_trades
        self.trades_per_signal = trades_per_signal

    def set_market_conditions(self, volatile_threshold: float, confidence_threshold: float,
                            min_confidence: float):
        """Set market condition thresholds"""
        self.volatile_threshold = volatile_threshold
        self.confidence_threshold = confidence_threshold
        self.min_confidence = min_confidence

    def log(self, message: str, icon: str = "📊"):
        """Enhanced logging with icons and timestamps"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{icon} [{timestamp}] {message}")

    async def connect(self):
        """Establish WebSocket connection"""
        try:
            self.websocket = await websockets.connect(
                f"wss://ws.derivws.com/websockets/v3?app_id={self.app_id}"
            )
            self.log("Connected to Deriv WebSocket", "🔗")

            # Authorize connection
            auth_msg = {"authorize": self.token}
            await self.websocket.send(json.dumps(auth_msg))
            response = await self.websocket.recv()
            auth_data = json.loads(response)

            if "error" in auth_data:
                self.log(f"Authorization failed: {auth_data['error']['message']}", "❌")
                return False

            self.balance = auth_data.get("authorize", {}).get("balance", 0)
            self.log(f"Authorized successfully. Balance: ${self.balance:.2f}", "✅")
            return True

        except Exception as e:
            self.log(f"Connection error: {str(e)}", "❌")
            return False

    async def get_historical_data(self, count: int = 100):
        """Fetch historical tick data"""
        try:
            ticks_history_msg = {
                "ticks_history": self.symbol,
                "count": count,
                "end": "latest",
                "style": "ticks"
            }

            await self.websocket.send(json.dumps(ticks_history_msg))
            response = await self.websocket.recv()
            data = json.loads(response)

            if "error" in data:
                self.log(f"Error fetching historical data: {data['error']['message']}", "❌")
                return False

            history = data["history"]
            df = pd.DataFrame({
                'timestamp': pd.to_datetime(history["times"], unit='s'),
                'price': history["prices"]
            })
            df.set_index('timestamp', inplace=True)
            df['returns'] = df['price'].pct_change()
            df['volatility'] = df['returns'].rolling(10).std()

            self.historical_data = df.dropna()
            self.log(f"Loaded {len(self.historical_data)} historical ticks", "📈")
            return True

        except Exception as e:
            self.log(f"Error loading historical data: {str(e)}", "❌")
            return False

    def prepare_features(self, data: pd.DataFrame):
        """Prepare features for SARIMAX model"""
        features = pd.DataFrame(index=data.index)
        features['price'] = data['price']
        features['returns'] = data['returns']
        features['volatility'] = data['volatility']
        features['price_lag1'] = data['price'].shift(1)
        features['price_lag2'] = data['price'].shift(2)
        features['returns_lag1'] = data['returns'].shift(1)
        features['ma_5'] = data['price'].rolling(5).mean()
        features['ma_10'] = data['price'].rolling(10).mean()

        return features.dropna()

    def train_model(self):
        """Train SARIMAX model"""
        try:
            if len(self.historical_data) < 30:
                self.log("Insufficient data for training", "⚠️")
                return False

            features = self.prepare_features(self.historical_data)

            # Prepare exogenous variables
            exog_vars = ['returns', 'volatility', 'price_lag1', 'returns_lag1']
            exog_data = features[exog_vars].fillna(method='bfill')

            # Fit SARIMAX model
            self.model = SARIMAX(
                features['price'],
                exog=exog_data,
                order=(1, 1, 1),
                seasonal_order=(0, 0, 0, 0),
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            self.model = self.model.fit(disp=False)
            self.log("SARIMAX model trained successfully", "🤖")
            return True

        except Exception as e:
            self.log(f"Model training error: {str(e)}", "❌")
            return False

    def predict_next_tick(self):
        """Make prediction for next tick"""
        try:
            if self.model is None:
                return None, 0.0

            # Get latest data for exogenous variables
            latest_data = self.historical_data.tail(1)
            features = self.prepare_features(self.historical_data.tail(20))

            if len(features) == 0:
                return None, 0.0

            exog_vars = ['returns', 'volatility', 'price_lag1', 'returns_lag1']
            exog_latest = features[exog_vars].tail(1).fillna(method='bfill')

            # Make prediction
            forecast = self.model.forecast(steps=1, exog=exog_latest)
            prediction = forecast.iloc[0]

            # Calculate confidence based on model's standard error
            confidence_interval = self.model.get_forecast(steps=1, exog=exog_latest).conf_int()
            confidence = 1.0 - (confidence_interval.iloc[0, 1] - confidence_interval.iloc[0, 0]) / prediction
            confidence = max(0.0, min(1.0, abs(confidence)))

            return prediction, confidence

        except Exception as e:
            self.log(f"Prediction error: {str(e)}", "❌")
            return None, 0.0

    def analyze_market_condition(self):
        """Analyze current market volatility"""
        if len(self.historical_data) < 10:
            return "unknown"

        current_volatility = self.historical_data['volatility'].tail(1).iloc[0]
        avg_volatility = self.historical_data['volatility'].tail(20).mean()

        if current_volatility > avg_volatility * self.volatile_threshold:
            return "volatile"
        elif current_volatility < avg_volatility * 0.5:
            return "calm"
        else:
            return "normal"

    def should_trade(self, prediction: float, confidence: float, current_price: float):
        """Determine if we should place a trade"""
        if confidence < self.min_confidence:
            return None, "Low confidence"

        market_condition = self.analyze_market_condition()

        # Adjust confidence threshold based on market condition
        required_confidence = self.confidence_threshold
        if market_condition == "volatile":
            required_confidence += 0.1
        elif market_condition == "calm":
            required_confidence -= 0.05

        if confidence < required_confidence:
            return None, f"Confidence {confidence:.3f} below threshold for {market_condition} market"

        # Determine trade direction
        price_diff = (prediction - current_price) / current_price
        print(price_diff)
        if abs(price_diff) < 0.0000009:  # Too small movement
            return None, "Predicted movement too small"

        direction = "CALL" if price_diff > 0 else "PUT"
        return direction, f"{market_condition} market, confidence: {confidence:.3f}"

    async def place_trade(self, direction: str, reason: str):
        """Place a binary options trade"""
        try:
            trade_msg = {
                "buy": 1,
                "price": self.stake_amount,
                "parameters": {
                    "contract_type": direction,
                    "symbol": self.symbol,
                    "duration": self.duration,
                    "duration_unit": self.duration_unit,
"currency":"USD",
"basis":"stake",
"amount":self.stake_amount
                }
            }

            await self.websocket.send(json.dumps(trade_msg))
            response = await self.websocket.recv()
            trade_data = json.loads(response)

            if "error" in trade_data:
                self.log(f"Trade placement failed: {trade_data['error']['message']}", "❌")
                return False

            contract_id = trade_data["buy"]["contract_id"]
            buy_price = trade_data["buy"]["buy_price"]

            trade_info = {
                'contract_id': contract_id,
                'direction': direction,
                'stake': self.stake_amount,
                'buy_price': buy_price,
                'entry_price': self.current_tick,
                'timestamp': datetime.now(),
                'reason': reason,
                'status': 'active'
            }

            self.active_trades.append(trade_info)
            self.trade_count += 1

            self.log(f"Trade #{self.trade_count} placed: {direction} ${self.stake_amount} | {reason}", "🚀")
            return True

        except Exception as e:
            self.log(f"Trade placement error: {str(e)}", "❌")
            return False

    async def check_active_trades(self):
        """Check status of active trades"""
        for trade in self.active_trades[:]:
            try:
                proposal_msg = {
                    "proposal_open_contract": 1,
                    "contract_id": trade['contract_id']
                }

                await self.websocket.send(json.dumps(proposal_msg))
                response = await self.websocket.recv()
                contract_data = json.loads(response)

                if "error" in contract_data:
                    continue

                contract = contract_data.get("proposal_open_contract", {})

                if contract.get("is_sold"):
                    # Trade completed
                    profit = contract.get("profit", 0)
                    sell_price = contract.get("sell_price", 0)

                    trade['profit'] = profit
                    trade['sell_price'] = sell_price
                    trade['exit_price'] = self.current_tick
                    trade['status'] = 'completed'

                    self.active_trades.remove(trade)
                    self.trade_history.append(trade)

                    self.total_profit += profit
                    if profit > 0:
                        self.wins += 1
                        result_icon = "✅"
                    else:
                        self.losses += 1
                        result_icon = "❌"

                    # Calculate prediction accuracy
                    entry_price = trade['entry_price']
                    exit_price = trade['exit_price']
                    predicted_direction = trade['direction']
                    actual_direction = "CALL" if exit_price > entry_price else "PUT"

                    accuracy = "✓" if predicted_direction == actual_direction else "✗"

                    self.log(f"Trade completed {result_icon} | Profit: ${profit:.2f} | "
                           f"Prediction: {accuracy} | Entry: {entry_price:.5f} → Exit: {exit_price:.5f}", "📊")

                    await self.update_balance()

            except Exception as e:
                self.log(f"Error checking trade {trade['contract_id']}: {str(e)}", "❌")

    async def update_balance(self):
        """Update current balance"""
        try:
            balance_msg = {"balance": 1}
            await self.websocket.send(json.dumps(balance_msg))
            response = await self.websocket.recv()
            balance_data = json.loads(response)

            if "balance" in balance_data:
                self.balance = balance_data["balance"]["balance"]

        except Exception as e:
            self.log(f"Balance update error: {str(e)}", "❌")

    def print_statistics(self):
        """Print comprehensive trading statistics"""
        win_rate = (self.wins / max(1, self.wins + self.losses)) * 100
        avg_profit = self.total_profit / max(1, len(self.trade_history))

        print("\n" + "="*60)
        print("📊 TRADING STATISTICS")
        print("="*60)
        print(f"💰 Current Balance: ${self.balance:.2f}")
        print(f"📈 Total Profit: ${self.total_profit:.2f}")
        print(f"🎯 Trades: {len(self.trade_history)} (Active: {len(self.active_trades)})")
        print(f"✅ Wins: {self.wins} | ❌ Losses: {self.losses}")
        print(f"📊 Win Rate: {win_rate:.1f}%")
        print(f"💵 Avg Profit per Trade: ${avg_profit:.2f}")
        print(f"🔄 Current Tick: {self.current_tick:.5f}" if self.current_tick else "")
        print("="*60 + "\n")

    async def subscribe_to_ticks(self):
        """Subscribe to real-time tick data"""
        try:
            tick_msg = {"ticks": self.symbol, "subscribe": 1}
            await self.websocket.send(json.dumps(tick_msg))
            self.log(f"Subscribed to {self.symbol} ticks", "📡")

        except Exception as e:
            self.log(f"Tick subscription error: {str(e)}", "❌")

    async def trading_loop(self):
        """Main trading loop"""
        self.is_trading = True
        self.log("Starting trading loop", "🚀")

        while self.is_trading and self.trade_count < self.max_trades:
            try:
                # Receive tick data
                response = await self.websocket.recv()
                data = json.loads(response)

                if "tick" in data:
                    tick_data = data["tick"]
                    self.current_tick = tick_data["quote"]
                    timestamp = pd.to_datetime(tick_data["epoch"], unit='s')

                    # Update historical data
                    new_row = pd.DataFrame({
                        'price': [self.current_tick],
                        'returns': [np.nan],
                        'volatility': [np.nan]
                    }, index=[timestamp])

                    if not self.historical_data.empty:
                        new_row['returns'] = (self.current_tick - self.historical_data['price'].iloc[-1]) / self.historical_data['price'].iloc[-1]

                    self.historical_data = pd.concat([self.historical_data, new_row])
                    self.historical_data = self.historical_data.tail(200)  # Keep last 200 ticks

                    # Recalculate volatility
                    self.historical_data['volatility'] = self.historical_data['returns'].rolling(10).std()

                    # Check active trades
                    await self.check_active_trades()

                    # Only trade if no active trades (prevent overlapping)
                    if len(self.active_trades) == 0 and len(self.historical_data) > 30:
                        # Retrain model periodically
                        if len(self.historical_data) % 50 == 0:
                            self.train_model()

                        # Make prediction
                        prediction, confidence = self.predict_next_tick()

                        if prediction is not None:
                            # Decide on trade
                            direction, reason = self.should_trade(prediction, confidence, self.current_tick)

                            if direction == "PUT":
                                # Place multiple trades per signal if configured
                                for i in range(self.trades_per_signal):
                                    if self.trade_count < self.max_trades:
                                        await self.place_trade(direction, reason)
                                        await asyncio.sleep(0.1)  # Small delay between trades

                            self.log(f"Prediction: {prediction:.5f} | Current: {self.current_tick:.5f} | "
                                   f"Confidence: {confidence:.3f} | Action: {direction or 'HOLD'}", "🔮")

                    # Print statistics every 10 ticks
                    if len(self.historical_data) % 10 == 0:
                        self.print_statistics()

                await asyncio.sleep(0.01)  # Small delay to prevent overwhelming

            except websockets.exceptions.ConnectionClosed:
                self.log("WebSocket connection closed", "⚠️")
                break
            except Exception as e:
                self.log(f"Trading loop error: {str(e)}", "❌")
                await asyncio.sleep(1)

    async def run(self):
        """Main bot execution"""
        self.log("Initializing Deriv Trading Bot", "🤖")

        # Connect to API
        if not await self.connect():
            return

        # Load historical data and train model
        if not await self.get_historical_data():
            return

        if not self.train_model():
            return

        # Subscribe to real-time ticks
        await self.subscribe_to_ticks()

        # Start trading
        await self.trading_loop()

        # Final statistics
        self.print_statistics()
        self.log("Trading bot stopped", "🛑")

# Usage Example
async def main():
    bot = DerivTradingBot()

    # Set connection parameters
    bot.set_connection_params("40003", "1C1Qo1372JjoJ1w")

    # Set trading parameters
    bot.set_trading_params(
        symbol="R_50",  # Synthetic index
        duration=1,  # 5 ticks
        duration_unit="t",  # ticks
        stake_amount=1000.0,  # $1 per trade
        max_trades=100,  # Maximum number of trades
        trades_per_signal=50  # Trades per signal
    )

    # Set market condition parameters
    bot.set_market_conditions(
        volatile_threshold=1.5,  # Volatility threshold multiplier
        confidence_threshold=0.6,  # Minimum confidence for normal market
        min_confidence=0.5  # Absolute minimum confidence
    )

    # Run the bot
    await bot.run()

if __name__ == "__main__":
    await (main())

🤖 [2025-06-08 11:42:55.721] Initializing Deriv Trading Bot
🔗 [2025-06-08 11:42:56.337] Connected to Deriv WebSocket
✅ [2025-06-08 11:42:56.536] Authorized successfully. Balance: $86201.48
📈 [2025-06-08 11:42:56.673] Loaded 90 historical ticks
🤖 [2025-06-08 11:42:57.503] SARIMAX model trained successfully
📡 [2025-06-08 11:42:57.503] Subscribed to R_50 ticks
🚀 [2025-06-08 11:42:57.503] Starting trading loop
-1.1072945867876141e-07
🔮 [2025-06-08 11:42:58.255] Prediction: 136.81918 | Current: 136.81920 | Confidence: 1.000 | Action: HOLD
-3.7171377657813675e-07
🔮 [2025-06-08 11:43:00.282] Prediction: 136.76825 | Current: 136.76830 | Confidence: 1.000 | Action: HOLD
-1.2884625925835813e-07
🔮 [2025-06-08 11:43:02.250] Prediction: 136.76478 | Current: 136.76480 | Confidence: 1.000 | Action: HOLD
1.4978096946811112e-07
🔮 [2025-06-08 11:43:04.311] Prediction: 136.79342 | Current: 136.79340 | Confidence: 1.000 | Action: HOLD
-2.5535972419529346e-08
🔮 [2025-06-08 11:43:06.383] Prediction: 136.8012

In [ ]:

import asyncio
import websockets
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class DerivTradingBot:
    def __init__(self):
        # Connection settings
        self.ws_url = "wss://ws.derivws.com/websockets/v3?app_id=40003"
        self.token = "1C1Qo1372JjoJ1w"
        self.websocket = None

        # Trading parameters (set these before running)
        self.symbol = "R_50"  # Default symbol
        self.stake_amount = 1.0
        self.duration = 5
        self.duration_unit = "t"  # ticks
        self.max_trades_per_signal = 3
        self.total_trade_limit = 100

        # Market condition thresholds
        self.volatile_threshold = 0.7
        self.normal_threshold = 0.6
        self.calm_threshold = 0.5

        # Data storage
        self.tick_data = pd.DataFrame()
        self.trade_history = []
        self.current_trade = None
        self.balance = 0.0
        self.trade_count = 0
        self.active_trades = 0

        # Model
        self.model = None
        self.scaler = StandardScaler()

        print(f"🤖 {self.get_timestamp()} Trading Bot Initialized")

    def get_timestamp(self):
        return datetime.now().strftime("%H:%M:%S")

    async def connect(self):
        """Connect to Deriv WebSocket"""
        try:
            self.websocket = await websockets.connect(self.ws_url)
            await self.authorize()
            print(f"✅ {self.get_timestamp()} Connected to Deriv API")
        except Exception as e:
            print(f"❌ {self.get_timestamp()} Connection failed: {e}")

    async def authorize(self):
        """Authorize with API token"""
        auth_request = {"authorize": self.token}
        await self.websocket.send(json.dumps(auth_request))
        response = await self.websocket.recv()
        data = json.loads(response)

        if 'authorize' in data:
            print(f"🔑 {self.get_timestamp()} Authorized successfully")
            self.balance = data['authorize'].get('balance', 0)
            print(f"💰 {self.get_timestamp()} Current balance: ${self.balance}")
        else:
            print(f"❌ {self.get_timestamp()} Authorization failed")

    async def get_historical_data(self, count=1000):
        """Get historical tick data"""
        request = {
            "ticks_history": self.symbol,
            "adjust_start_time": 1,
            "count": count,
            "end": "latest",
            "start": 1,
            "style": "ticks"
        }

        await self.websocket.send(json.dumps(request))
        response = await self.websocket.recv()
        data = json.loads(response)

        if 'history' in data:
            prices = data['history']['prices']
            times = data['history']['times']

            df = pd.DataFrame({
                'timestamp': pd.to_datetime(times, unit='s'),
                'price': prices
            })
            df.set_index('timestamp', inplace=True)
            self.tick_data = df
            print(f"📊 {self.get_timestamp()} Historical data loaded: {len(df)} ticks")
            return True
        return False

    async def subscribe_ticks(self):
        """Subscribe to real-time tick stream"""
        request = {"ticks": self.symbol, "subscribe": 1}
        await self.websocket.send(json.dumps(request))
        print(f"📡 {self.get_timestamp()} Subscribed to {self.symbol} ticks")

    def prepare_features(self, data):
        """Prepare features for SARIMAX model"""
        if len(data) < 20:
            return None

        df = data.copy()

        # Price features
        df['returns'] = df['price'].pct_change()
        df['sma_5'] = df['price'].rolling(5).mean()
        df['sma_10'] = df['price'].rolling(10).mean()
        df['volatility'] = df['returns'].rolling(10).std()
        df['momentum'] = df['price'] - df['price'].shift(5)

        # Technical indicators
        df['rsi'] = self.calculate_rsi(df['price'])
        df['bb_upper'], df['bb_lower'] = self.calculate_bollinger_bands(df['price'])

        df.dropna(inplace=True)
        return df

    def calculate_rsi(self, prices, period=14):
        """Calculate RSI"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def calculate_bollinger_bands(self, prices, period=20, std_dev=2):
        """Calculate Bollinger Bands"""
        sma = prices.rolling(period).mean()
        std = prices.rolling(period).std()
        upper = sma + (std * std_dev)
        lower = sma - (std * std_dev)
        return upper, lower

    def train_sarimax_model(self, data):
        """Train SARIMAX model"""
        try:
            if len(data) < 50:
                return False

            features = self.prepare_features(data)
            if features is None or len(features) < 30:
                return False

            # Prepare exogenous variables
            exog_vars = ['returns', 'volatility', 'momentum', 'rsi']
            exog = features[exog_vars].fillna(0)
            target = features['price']

            # Fit SARIMAX model
            self.model = SARIMAX(
                target,
                exog=exog,
                order=(1, 1, 1),
                seasonal_order=(0, 0, 0, 0),
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            self.model_fit = self.model.fit(disp=False)
            print(f"🧠 {self.get_timestamp()} SARIMAX model trained successfully")
            return True

        except Exception as e:
            print(f"❌ {self.get_timestamp()} Model training failed: {e}")
            return False

    def get_prediction_confidence(self, data):
        """Get prediction with confidence interval"""
        try:
            if self.model_fit is None:
                return None, 0

            features = self.prepare_features(data)
            if features is None:
                return None, 0

            exog_vars = ['returns', 'volatility', 'momentum', 'rsi']
            exog = features[exog_vars].iloc[-1:].fillna(0)

            forecast = self.model_fit.forecast(steps=1, exog=exog)
            conf_int = self.model_fit.get_forecast(steps=1, exog=exog).conf_int()

            prediction = forecast.iloc[0]
            current_price = data['price'].iloc[-1]

            # Calculate confidence based on prediction interval width
            interval_width = conf_int.iloc[0, 1] - conf_int.iloc[0, 0]
            confidence = max(0, min(1, 1 - (interval_width / current_price)))

            return prediction, confidence

        except Exception as e:
            print(f"⚠️ {self.get_timestamp()} Prediction error: {e}")
            return None, 0

    def determine_trade_signal(self, current_price, prediction, confidence):
        """Determine trading signal based on prediction and market conditions"""
        if prediction is None or confidence < 0.3:
            return "HOLD", 0

        price_change = (prediction - current_price) / current_price
        volatility = self.tick_data['price'].rolling(10).std().iloc[-1] / current_price

        # Adjust thresholds based on volatility
        if volatility > 0.002:  # High volatility
            threshold = self.volatile_threshold
            signal_strength = "VOLATILE"
        elif volatility > 0.001:  # Normal volatility
            threshold = self.normal_threshold
            signal_strength = "NORMAL"
        else:  # Low volatility
            threshold = self.calm_threshold
            signal_strength = "CALM"

        print(f"📈 {self.get_timestamp()} Market: {signal_strength}, Confidence: {confidence:.3f}, Change: {price_change:.4f}")

        if confidence >= threshold:
            if price_change > 0.0005:  # 0.05% threshold
                return "BUY", confidence
            elif price_change < -0.0005:
                return "SELL", confidence

        return "HOLD", confidence

    async def place_trade(self, signal, confidence):
        """Place a trade based on signal"""
        if self.active_trades >= self.max_trades_per_signal:
            print(f"⏸️ {self.get_timestamp()} Max active trades reached")
            return False

        if self.trade_count >= self.total_trade_limit:
            print(f"🛑 {self.get_timestamp()} Trade limit reached")
            return False

        contract_type = "CALL" if signal == "BUY" else "PUT"

        # Risk management: reduce stake on losing streaks
        recent_trades = self.trade_history[-5:] if len(self.trade_history) >= 5 else self.trade_history
        if recent_trades:
            win_rate = sum(1 for t in recent_trades if t['result'] == 'WIN') / len(recent_trades)
            adjusted_stake = self.stake_amount * max(0.5, win_rate * 2)
        else:
            adjusted_stake = self.stake_amount

        trade_request = {
            "buy": 1,
            "price": adjusted_stake,
            "parameters": {
                "contract_type": contract_type,
                "symbol": self.symbol,
                "duration": self.duration,
                "duration_unit": self.duration_unit,
"currency":"USD",
"basis":"stake",

"amount":adjusted_stake
            }
        }

        try:
            await self.websocket.send(json.dumps(trade_request))
            response = await self.websocket.recv()
            data = json.loads(response)

            if 'buy' in data:
                self.current_trade = {
                    'id': data['buy']['contract_id'],
                    'type': contract_type,
                    'stake': adjusted_stake,
                    'entry_price': self.tick_data['price'].iloc[-1],
                    'prediction': None,
                    'confidence': confidence,
                    'timestamp': datetime.now()
                }
                self.active_trades += 1
                self.trade_count += 1

                print(f"🚀 {self.get_timestamp()} Trade placed: {contract_type} ${adjusted_stake} (Confidence: {confidence:.3f})")
                await self.monitor_trade()
                return True

        except Exception as e:
            print(f"❌ {self.get_timestamp()} Trade placement failed: {e}")

        return False

    async def monitor_trade(self):
        """Monitor current trade"""
        if not self.current_trade:
            return

        # Subscribe to trade updates
        request = {
            "proposal_open_contract": 1,
            "contract_id": self.current_trade['id'],
            "subscribe": 1
        }

        await self.websocket.send(json.dumps(request))

    def process_trade_result(self, trade_data):
        """Process completed trade result"""
        if not self.current_trade:
            return

        result = "WIN" if trade_data.get('is_sold', 0) == 1 and trade_data.get('sell_price', 0) > self.current_trade['stake'] else "LOSS"
        payout = trade_data.get('sell_price', 0)
        profit = payout - self.current_trade['stake']

        trade_record = {
            'id': self.current_trade['id'],
            'type': self.current_trade['type'],
            'stake': self.current_trade['stake'],
            'payout': payout,
            'profit': profit,
            'result': result,
            'confidence': self.current_trade['confidence'],
            'entry_price': self.current_trade['entry_price'],
            'exit_price': self.tick_data['price'].iloc[-1] if not self.tick_data.empty else 0,
            'timestamp': self.current_trade['timestamp']
        }

        self.trade_history.append(trade_record)
        self.balance += profit
        self.active_trades -= 1

        icon = "✅" if result == "WIN" else "❌"
        print(f"{icon} {self.get_timestamp()} Trade completed: {result} ${profit:.2f}")

        self.current_trade = None
        self.print_statistics()

    def print_statistics(self):
        """Print trading statistics"""
        if not self.trade_history:
            return

        total_trades = len(self.trade_history)
        wins = sum(1 for t in self.trade_history if t['result'] == 'WIN')
        losses = total_trades - wins
        win_rate = (wins / total_trades) * 100

        total_profit = sum(t['profit'] for t in self.trade_history)
        avg_profit = total_profit / total_trades

        print(f"\n📊 {self.get_timestamp()} TRADING STATISTICS")
        print(f"💰 Balance: ${self.balance:.2f}")
        print(f"📈 Total Trades: {total_trades}")
        print(f"✅ Wins: {wins} ({win_rate:.1f}%)")
        print(f"❌ Losses: {losses}")
        print(f"💵 Total P&L: ${total_profit:.2f}")
        print(f"📊 Avg P&L: ${avg_profit:.2f}")
        print("-" * 40)

    async def process_tick(self, tick_data):
        """Process incoming tick data"""
        timestamp = pd.to_datetime(tick_data['epoch'], unit='s')
        price = tick_data['quote']

        # Add new tick to data
        new_tick = pd.DataFrame({'price': [price]}, index=[timestamp])
        self.tick_data = pd.concat([self.tick_data, new_tick]).tail(200)  # Keep last 200 ticks

        # Skip trading if there's an active trade
        if self.current_trade:
            return

        # Train model periodically
        if len(self.tick_data) >= 50 and len(self.tick_data) % 20 == 0:
            self.train_sarimax_model(self.tick_data)

        # Get prediction and make trading decision
        if self.model_fit and len(self.tick_data) >= 30:
            prediction, confidence = self.get_prediction_confidence(self.tick_data)
            signal, signal_confidence = self.determine_trade_signal(price, prediction, confidence)

            if signal in ["BUY", "SELL"]:
                await self.place_trade(signal, signal_confidence)

    async def run(self):
        """Main trading loop"""
        await self.connect()

        # Get initial historical data
        if await self.get_historical_data():
            self.train_sarimax_model(self.tick_data)

        # Subscribe to tick stream
        await self.subscribe_ticks()

        print(f"🤖 {self.get_timestamp()} Trading bot started")

        try:
            while True:
                response = await self.websocket.recv()
                data = json.loads(response)

                if 'tick' in data:
                    await self.process_tick(data['tick'])
                elif 'proposal_open_contract' in data:
                    contract_data = data['proposal_open_contract']
                    if contract_data.get('is_expired') or contract_data.get('is_sold'):
                        self.process_trade_result(contract_data)

        except Exception as e:
            print(f"❌ {self.get_timestamp()} Error in main loop: {e}")
        finally:
            if self.websocket:
                await self.websocket.close()

# Usage example
async def main():
    bot = DerivTradingBot()

    # Set trading parameters
    bot.symbol = "R_50"  # Volatility 50 Index
    bot.stake_amount = 1000.0
    bot.duration = 1
    bot.duration_unit = "t"
    bot.max_trades_per_signal = 50
    bot.total_trade_limit = 50

    # Set confidence thresholds
    bot.volatile_threshold = 0.65
    bot.normal_threshold = 0.55
    bot.calm_threshold = 0.45

    await bot.run()

if __name__ == "__main__":
    await (main())

🤖 16:34:24 Trading Bot Initialized
🔑 16:34:25 Authorized successfully
💰 16:34:25 Current balance: $39697.36
✅ 16:34:25 Connected to Deriv API
📊 16:34:25 Historical data loaded: 1000 ticks
🧠 16:34:30 SARIMAX model trained successfully
📡 16:34:30 Subscribed to R_50 ticks
🤖 16:34:30 Trading bot started
🧠 16:34:32 SARIMAX model trained successfully
📈 16:34:32 Market: CALM, Confidence: 1.000, Change: -0.0000
🧠 16:34:35 SARIMAX model trained successfully
📈 16:34:35 Market: CALM, Confidence: 1.000, Change: 0.0000
🧠 16:34:36 SARIMAX model trained successfully
📈 16:34:36 Market: CALM, Confidence: 1.000, Change: 0.0000
🧠 16:34:38 SARIMAX model trained successfully
📈 16:34:38 Market: CALM, Confidence: 1.000, Change: -0.0000
🧠 16:34:40 SARIMAX model trained successfully
📈 16:34:40 Market: CALM, Confidence: 1.000, Change: 0.0000
🧠 16:34:42 SARIMAX model trained successfully
📈 16:34:42 Market: CALM, Confidence: 1.000, Change: 0.0000
🧠 16:34:44 SARIMAX model trained successfully
📈 16:34:44 Market: CA

CancelledError: 